In [24]:
import re
import collections
import random
import numpy as np
from tqdm import tqdm

In [2]:
def load_corpus():
    with open("./corpus.txt", "r") as corpus:
        for line in corpus:
            line = line.rstrip()
            if line != "":
                yield line

Task 1a.

In [27]:
def tokenize_sentence():
  for i,sentence in enumerate(load_corpus()):
      sentence = re.sub(r"[,.;:]", "", sentence)
      words = re.split(r"\s+", sentence)
      words.insert(0, "<s>")
      words.append("</s>")
      yield words
for i,sentence in enumerate(tokenize_sentence()):
        if i<5:
            print("sentence: ", i,": ",sentence)
        else: break

sentence:  0 :  ['<s>', 'the', 'relative', 'geographical', 'isolation', 'of', 'the', 'brandywine', 'population', 'makes', 'for', 'a', 'limited', 'choice', 'in', 'mating', '</s>']
sentence:  1 :  ['<s>', 'then', 'there', 'was', 'a', 'bad', 'delay', 'in', 'getting', 'mort', "lindsey's", '30-piece', 'orchestra', 'wedged', 'into', 'its', 'chairs', '</s>']
sentence:  2 :  ['<s>', 'if', 'he', 'instructs', 'them', 'in', 'how', 'to', 'evaluate', 'a', 'work', 'he', 'is', 'helping', 'them', 'to', 'achieve', 'their', 'own', 'identity', '</s>']
sentence:  3 :  ['<s>', 'all', 'your', 'wishful', 'thinking', "won't", 'change', 'that', '</s>']
sentence:  4 :  ['<s>', 'indeed', 'the', 'contrary', 'has', 'happened', '</s>']


Task 1b.

In [28]:
class LanguageModel:
  def __init__(self, corpus):
    self.unigram_counts = collections.Counter()
    self.bigram_counts = collections.Counter()
    self.trigram_counts = collections.Counter()

    for _, sentence in enumerate(corpus()):
      self.unigram_counts.update(sentence)
      for i in range(len(sentence) - 1):
        self.bigram_counts[(sentence[i], sentence[i + 1])] += 1
      for i in range(len(sentence) - 2):
        self.trigram_counts[(sentence[i], sentence[i + 1], sentence[i + 2])] += 1

    self.total_word_count = sum(self.unigram_counts.values())

  def unigram_distribution(self):
    unigram_probabilities = {}
    for word, count in self.unigram_counts.items():
      unigram_probabilities[word] = count / self.total_word_count

    return unigram_probabilities

  def bigram_distribution(self, w1):
    bigram_probabilities = {}
    for w2, count in self.bigram_counts.items():
      if w1 == w2[0]:
        bigram_probabilities[w2[1]] = count / self.unigram_counts[w1]

    return bigram_probabilities

  def trigram_distribution(self, w1, w2):
    trigram_probabilities = {}
    for w3, count in self.trigram_counts.items():
      if w1 == w3[0] and w2 == w3[1]:
        trigram_probabilities[w3[2]] = count / self.bigram_counts[(w1, w2)]
    return trigram_probabilities

language_model = LanguageModel(tokenize_sentence)
unigram_prob = language_model.unigram_distribution()
bigram_prob = language_model.bigram_distribution("present")
trigram_prob = language_model.trigram_distribution("raw","material")
print(bigram_prob)
print(trigram_prob)

{'generation': 0.003105590062111801, 'different': 0.003105590062111801, 'to': 0.018633540372670808, 'crew': 0.003105590062111801, 'emotion': 0.003105590062111801, 'information': 0.006211180124223602, "it's": 0.003105590062111801, 'at': 0.018633540372670808, 'was': 0.003105590062111801, 'difficulties': 0.003105590062111801, 'there': 0.003105590062111801, 'my': 0.003105590062111801, 'in': 0.06521739130434782, 'themselves': 0.009316770186335404, 'yankees': 0.003105590062111801, 'trained': 0.003105590062111801, 'enemies': 0.003105590062111801, 'level': 0.012422360248447204, 'conditions': 0.006211180124223602, 'artless': 0.003105590062111801, 'and': 0.049689440993788817, 'last': 0.003105590062111801, 'simultaneously': 0.003105590062111801, 'their': 0.003105590062111801, 'as': 0.003105590062111801, 'all': 0.006211180124223602, 'era': 0.003105590062111801, 'sufficient': 0.003105590062111801, 'widespread': 0.003105590062111801, 'project': 0.003105590062111801, 'pool': 0.003105590062111801, 'da

Task 1c.

In [29]:
print("Number of parameters in unigram model: ",np.array( list(language_model.unigram_counts.keys())).shape)
print("Number of parameters in bigram model: ",np.array( list(language_model.bigram_counts.keys())).shape)
print("Number of parameters in trigram model: ",np.array( list(language_model.trigram_counts.keys())).shape)

Number of parameters in unigram model:  (47421,)
Number of parameters in bigram model:  (383665, 2)
Number of parameters in trigram model:  (689309, 3)


The number of parameters of the unigram, bigram and trigram distributions scales with the number of different words in the corpus:
Unigram distribution: The unigram distribution has one parameter for each pair of words in the corpus. For a corpus with N different words, the bigram distribution has N parameters.
Bigram distribution: The bigram distribution has one parameter for each pair of words in the corpus. For a corpus with N different words, the bigram distribution has N * N parameters.
Trigram distribution: The trigram distribution has one parameter for each triple of words in the corpus. For a corpus with N different words, the trigram distribution has N * N * N parameters.

Task 2a.

In [30]:
def Sample(language_model, distribution, previous_word="<s>"):
  if distribution== "unigram":
    unigram_prob = language_model.unigram_distribution()
    sample =random.choices(list(unigram_prob.keys()), weights=list(unigram_prob.values()))[0]
  elif distribution== "bigram":
    previous_word= "<s>"
    bigram_prob = language_model.bigram_distribution(previous_word)
    sample = random.choices(list(bigram_prob.keys()), weights=list(bigram_prob.values()))[0]
  elif distribution =="trigram":
    if previous_word == "<s>": previous_word = ("<s>", "</s>")
    trigram_prob=language_model.trigram_distribution(previous_word[0], previous_word[1])
    sample=random.choices(list(trigram_prob.keys()), weights=list(trigram_prob.values()))[0]
  return sample

# sample = sample(language_model, "unigram")
# print(sample)
sample = Sample(language_model, "bigram")
print(sample)

she


Task 2b.

In [62]:
def generate_sentence_unigram(language_model,max_sentance_length=100):
      sentence = []
      sentence_length=0
      while True:
        word = Sample(language_model, "unigram")
        if word == "</s>" or sentence_length==max_sentance_length:
          break
        sentence.append(word)
        sentence_length+=1
      return " ".join(sentence)


def generate_sentence_bigram(language_model,max_sentance_length=100):
      sentence = []
      sentence_length=0
      previous_word = "<s>"
      while True:
        word = Sample(language_model, "bigram", previous_word)
        if word == "</s>" or sentence_length==max_sentance_length:
          break
        sentence.append(word)
        previous_word = word
        sentence_length+=1
      return " ".join(sentence)


def generate_sentence_trigram(language_model,max_sentance_length=100):
      sentence = []
      sentence_length=0
      previous_two_words = ("<s>", Sample(language_model, "unigram"))
      while True:
        word = Sample(language_model, "trigram", previous_two_words)
        # if word == "</s>" or sentence_length==max_sentance_length:
        if word == "</s>":
          break
        sentence.append(word)
        previous_two_words = (previous_two_words[1], word)
        sentence_length+=1
      return " ".join(sentence)


Task 2c.

In [44]:
unigram_sentence = generate_sentence_unigram(language_model)
unigram_sentence

'on the genuine brown by not found of of'

The unigram generator only considers the frequency of individual words, and it does not consider the relationships between words or order.

In [45]:
bigram_sentence = generate_sentence_bigram(language_model,100)
bigram_sentence

"what the drunk pete he most both let it the if notwithstanding in a the this the cady but we'll to at the because we success in that the to others so she mrs maybe he their he other the this st while when a i a only the and welsh when gone if the prednisone indeed as the by the the the can or now one a a but harris he by as by pels that the to another in he he it a to he and it styka we i there with instead mrs a frog-marched it in"

Bigram generator only considers the frequency of pairs of words, and it does not consider the larger context of the sentence. Generator tend to generate very large text

In [67]:
trigram_sentence = generate_sentence_trigram(language_model,100)
trigram_sentence

'were not however find himself forced into a faith which is easily installed'

This the most coherent of the three sentences. This is because the trigram generator considers the frequency of triples of words, which allows it to capture more of the context of the sentence.